In [54]:
import numpy as np

from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

In [67]:
def gbdt_lr_train_test(libsvmFileName):
    #GBDT
    start = time.clock()
    train_df, test_df = train_test_split(example, train_size = 0.8)
    X_train = train_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_train = train_df['Grant.Status']
    X_test = test_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_test = test_df['Grant.Status']
    gbclf = GradientBoostingClassifier(n_estimators=30, max_depth=4, verbose=0)
    tuned_parameter = [{'n_estimators':[30, 40, 50, 60], 'max_depth':[3, 4, 5, 6], 'max_features':[0.5,0.7,0.9]}]
    gs_clf = GridSearchCV(gbclf, tuned_parameter, cv=5, scoring='roc_auc')
    gs_clf.fit(X_train, y_train)
    print('best parameters set found: ')
    print(gs_clf.best_params_)
    y_pred_gbdt = gs_clf.predict_proba(X_test)[:, 1]
    gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)
    print('gbdt auc: %.5f' % gbdt_auc)
    gbclf.fit(X_train,y_train)
    X_train_leaves = gbclf.apply(X_train)[:,:,0]
    (train_rows, cols) = X_train_leaves.shape
    gbclf.fit(X_test,y_test)
    X_test_leaves = gbclf.apply(X_test)[:,:,0]
    gbdtenc = OneHotEncoder()
    X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))
    #GBDT+LR
    lr = LogisticRegression()
    lr.fit(X_trans[:train_rows, :], y_train)
    y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_trans[:train_rows, :], X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_trans[train_rows:, :], X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('GBDT time taken: ', f_time)
    #XGB
    start = time.clock()
    xgb=XGBClassifier()
    xgb.fit(X_train,y_train)
    Y_pred_xgb=xgb.predict(X_test)
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    
    print('XGB auc : ',xgb_auc)
    f_time =time.clock()-start
    print('XGB time taken: ', f_time)
    #lightGBM
    '''start = time.clock()
    
    print('lightGBM time taken: ', time.clock()-start)'''

In [3]:
example = pd.read_csv('example.csv')

In [68]:
gbdt_lr_train_test(example)

best parameters set found: 
{'max_depth': 6, 'max_features': 0.7, 'n_estimators': 60}
gbdt auc: 0.93692
gbdt+lr auc 1: 0.78679
gbdt+lr auc 2: 0.55240
GBDT time taken:  104.3263780000002
XGB auc :  0.855922788101
XGB time taken:  0.7697490000000471


In [69]:
gbdt_lr_train_test(example)

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 60}
gbdt auc: 0.94193
gbdt+lr auc 1: 0.78375
gbdt+lr auc 2: 0.58599
GBDT time taken:  103.21583099999998
XGB auc :  0.860663462535
XGB time taken:  0.7435100000002421


In [70]:
gbdt_lr_train_test(example)

best parameters set found: 
{'max_depth': 6, 'max_features': 0.7, 'n_estimators': 60}
gbdt auc: 0.94117
gbdt+lr auc 1: 0.60895
gbdt+lr auc 2: 0.56951
GBDT time taken:  108.03956199999993
XGB auc :  0.865814311764
XGB time taken:  0.742572999999993
